In [ ]:
# Common imports and settings
import os
from IPython.display import Markdown
import pandas as pd
import xarray as xr
import hvplot.xarray
import cartopy.crs as ccrs

# Datacube
import datacube
from datacube.utils import masking
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map

os.environ['USE_PYGEOS'] = '0'
pd.set_option("display.max_rows", None)


In [ ]:
dc = datacube.Datacube()

In [ ]:
display(Markdown(f'# Products list:'))

products = dc.list_products()  # Pandas DataFrame
products

In [ ]:
measurements = dc.list_measurements()  # Pandas DataFrame, all products
measurements

In [ ]:
# latitude = (-1.9997527,1)
# longitude = (-92.029, -88.9707667)
latitude = (-0.75,-0.5)
longitude = (-90.529, -90.25)

display_map(longitude, latitude)

In [ ]:
# A standard datacube.load() call.
# This may take a few minutes while the data are loaded into JupyterLab (so choose a small area and time range).

data = dc.load(
    product = "io_lulc_annual_v02",
    latitude = latitude,
    longitude = longitude,
    time = ("2017-01-01", "2017-12-31"),
    group_by = 'solar_day',                    # Group by day method
    output_crs = 'EPSG:32715',                  # Output CRS
    resolution = (10, -10),                     # Resolution
)

display(data)

display(f'Number of MiB: {data.nbytes/2**20}')


In [ ]:
# Mask by nodata

# Under the hood: data != data.nodata -> bool
# Applies to each variable in an xarray.Dataset (including any bit-masks)
valid_mask = masking.valid_data_mask(data.land_cover_class)

In [ ]:

# Use numpy.where() to apply a mask array to measurement arrays
valid_data = data.where(valid_mask)  # Default: Where False replace with NaN -> convert dtype to float64

In [ ]:
band = "land_cover_class"
# Plot the data on a basemap
valid_data[band].isel(time=0).hvplot.image("x","y",
  geo=True,
  crs=ccrs.epsg(valid_data.attrs['crs'].split(':')[-1]),
  tiles='OSM',
  frame_width=700,
  frame_height=500,
  title=f'{band} Data on Basemap',
  cmap='tab20',
)